In [2]:
import os
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Define custom dataset class
class AnemiaDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Load image filenames and their corresponding labels
        for filename in os.listdir(data_dir):
            if filename.endswith(('.jpg', '.jpeg', '.png')):  # Check for image files
                # Check if the image name starts with "Anemic" or "Non-Anemic"
                if filename.startswith("Anemic"):
                    label = 1  # Anemic
                elif filename.startswith("Non-Anemic"):
                    label = 0  # Non-Anemic
                else:
                    continue  # Skip images with unexpected filenames
                
                # Append the full path and label
                self.image_paths.append(os.path.join(data_dir, filename))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Load the image
        image = Image.open(img_path).convert('RGB')  # Ensure 3 channels (RGB)

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label

# Define the transform to resize and normalize images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
])

# Define the path to the data folder
dataset_dir = r".\anemia_detection\Palm"

# Create custom dataset
dataset = AnemiaDataset(data_dir=dataset_dir, transform=transform)

# Split dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [3]:
from torchvision import models

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer for binary classification (2 classes)
model.fc = nn.Linear(model.fc.in_features, 1)  # Output size = 1 for binary classification
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available


c:\projs\anemia_project\cudavenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\projs\anemia_project\cudavenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)



In [5]:
torch.cuda.is_available()

True

In [9]:
# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            outputs = outputs.squeeze()  # Remove extra dimensions for binary classification
            loss = criterion(outputs, labels.float())  # Compute loss
            loss.backward()  # Backpropagate

            # Update weights
            optimizer.step()

            # Compute accuracy
            predicted = torch.round(torch.sigmoid(outputs))  # Get predictions
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            running_loss += loss.item()

        # Calculate training accuracy and loss
        train_accuracy = correct / total
        train_loss = running_loss / len(train_loader)

        # Evaluate on validation data
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
                outputs = model(inputs)
                outputs = outputs.squeeze()

                # Compute loss and accuracy
                loss = criterion(outputs, labels.float())
                val_loss += loss.item()

                predicted = torch.round(torch.sigmoid(outputs))
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        val_loss = val_loss / len(val_loader)

        # Print results for this epoch
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

        # Save the best model based on validation accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pt',_use_new_zipfile_serialization=True,pickle_protocol=4)
            torch.save(model.state_dict(), 'best_model.pth',_use_new_zipfile_serialization=True,pickle_protocol=4)

    print("Training complete.")
    print(f"Best Validation Accuracy: {best_accuracy:.4f}")

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)


Epoch [1/10], Train Loss: 0.0093, Train Accuracy: 0.9981, Val Loss: 0.0478, Val Accuracy: 0.9815
Epoch [2/10], Train Loss: 0.0117, Train Accuracy: 0.9957, Val Loss: 0.0437, Val Accuracy: 0.9840
Epoch [3/10], Train Loss: 0.0154, Train Accuracy: 0.9948, Val Loss: 0.0437, Val Accuracy: 0.9827
Epoch [4/10], Train Loss: 0.0126, Train Accuracy: 0.9960, Val Loss: 0.0465, Val Accuracy: 0.9790
Epoch [5/10], Train Loss: 0.0091, Train Accuracy: 0.9985, Val Loss: 0.0454, Val Accuracy: 0.9840
Epoch [6/10], Train Loss: 0.0133, Train Accuracy: 0.9969, Val Loss: 0.0478, Val Accuracy: 0.9840
Epoch [7/10], Train Loss: 0.0124, Train Accuracy: 0.9963, Val Loss: 0.0475, Val Accuracy: 0.9815
Epoch [8/10], Train Loss: 0.0111, Train Accuracy: 0.9969, Val Loss: 0.0461, Val Accuracy: 0.9815
Epoch [9/10], Train Loss: 0.0096, Train Accuracy: 0.9978, Val Loss: 0.0472, Val Accuracy: 0.9815
Epoch [10/10], Train Loss: 0.0114, Train Accuracy: 0.9972, Val Loss: 0.0453, Val Accuracy: 0.9864
Training complete.
Best Valid

In [7]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
        outputs = model(inputs)
        outputs = outputs.squeeze()

        predicted = torch.round(torch.sigmoid(outputs))
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Final Validation Accuracy: {accuracy:.4f}")


C:\Users\rohan\AppData\Local\Temp\ipykernel_7524\2410240957.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Final Validation Accuracy: 0.9852


In [ ]:
'_____________________________________'

In [10]:
import torch
from torchvision import transforms, models
from PIL import Image
import os

# Function to load the trained model
def load_model(model_path='best_model.pth'):
    # Load pre-trained ResNet50 model and modify for binary classification
    model = models.resnet50(pretrained=True)
    model.fc = torch.nn.Linear(model.fc.in_features, 1)  # Output size = 1 for binary classification
    model.load_state_dict(torch.load(model_path))  # Load the saved weights
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Move to GPU if available
    model.eval()  # Set the model to evaluation mode
    return model

# Function to preprocess the image
def preprocess_image(image_path):
    # Define the transforms (same as used in training)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to 224x224
        transforms.ToTensor(),  # Convert the image to a Tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet stats
    ])

    # Open the image file and apply the transformations
    image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format
    image = transform(image)
    image = image.unsqueeze(0)  # Add a batch dimension
    return image

# Function to make predictions
def predict_anemia(model, image_path):
    # Preprocess the input image
    image = preprocess_image(image_path)
    
    # Move image to the same device as the model (GPU or CPU)
    image = image.to('cuda' if torch.cuda.is_available() else 'cpu')

    # Forward pass through the model to get the prediction
    with torch.no_grad():
        output = model(image)  # Output is a single value

    # Apply sigmoid to the output to get a probability
    probability = torch.sigmoid(output).item()  # Convert the output to a scalar probability

    # If the probability is greater than 0.5, we classify it as "Anemic" (1)
    if probability > 0.5:
        return "Anemic", probability
    else:
        return "Non-Anemic", probability

# Example usage of the functions
if __name__ == '__main__':
    image_path = r'C:\projs\anemia_project\Non-AnemicP-040 (5).png'

    # Load the trained model
    model = load_model()

    # Make prediction on the input image
    label, probability = predict_anemia(model, image_path)
    
    # Print the result
    print(f"The person is predicted to be: {label}")
    print(f"Prediction probability: {probability:.4f}")


c:\projs\anemia_project\cudavenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\projs\anemia_project\cudavenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\rohan\AppData\Local\Temp\ipykernel_7524\1087447240.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/

The person is predicted to be: Non-Anemic
Prediction probability: 0.0000
